In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense, BatchNormalization,Input
from tensorflow.keras.optimizers import Adam


In [ ]:
img_size = (255, 255)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)


In [ ]:
dataset_path="/kaggle/input/animal-dataset/dataset"
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
model = Sequential([
    Input(shape=(255, 255, 3)),
    Conv2D(250, (3, 3), activation='relu', padding="same"),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
    Conv2D(200, (3, 3), activation='relu', padding="same"),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dense(15, activation='softmax')  # Adjusted for 15 classes
])

In [ ]:
optimizer = Adam(learning_rate=0.0001)  # Custom learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random  # To pick a random image

# Get a random batch from the validation generator
batch_index = random.randint(0, len(validation_generator) - 1)  # Choose a random batch
images, labels = validation_generator[batch_index]  # Load the chosen batch

# Pick a random image from the batch
image_index = random.randint(0, len(images) - 1)  # Choose a random image within the batch
random_image = images[image_index]

# Display the random image
plt.imshow(random_image)
plt.axis('off')  # Remove axis for better visualization
plt.show()

# Predict the class for the chosen image
predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Get the predicted class name
class_indices = validation_generator.class_indices  # Class-to-index mapping
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse the mapping
predicted_class_name = index_to_class[predicted_classes[image_index]]

# Print the predicted class name
print(f"Predicted Class: {predicted_class_name}")